## Homework: Vector Search

## Embeddings

Qdrant uses [fastembed](https://github.com/qdrant/fastembed)
under the hood to turn text into vectors.

In [2]:
pip install fastembed

Note: you may need to restart the kernel to use updated packages.


## Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`.
Use the `'jinaai/jina-embeddings-v2-small-en'` model. 

You should get a numpy array of size 512.

What's the minimal value in this array?

* -0.51
* -0.11
* 0
* 0.51

In [40]:
from fastembed import TextEmbedding

In [41]:
model = TextEmbedding(model_name = 'jinaai/jina-embeddings-v2-small-en')

In [42]:
query = 'I just discovered the course. Can I join now?'

In [43]:
embeddings = list(model.embed(query))

In [44]:
embeddings_array = embeddings[0]

In [45]:
print("The dimension of the array is {} and the minmal value in the array is {}".format(len(embeddings_array) ,min(embeddings_array)))

The dimension of the array is 512 and the minmal value in the array is -0.11726373885183883


## Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the `norm` function:

```python
import numpy as np
np.linalg.norm(q)
```

Which means that we can simply compute the dot product between
two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

```python
q.dot(q)
```

## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query
and the vector for the document?

* 0.3
* 0.5
* 0.7
* 0.9


In [46]:
import numpy as np

In [48]:
doc = 'Can I still join the course after the start date?'

In [57]:
embeddings_doc = list(model.embed(doc))
embeddings_array_doc = embeddings_doc[0]

In [58]:
len(embeddings_array_doc) ,min(embeddings_array_doc)

(512, -0.12396320482168117)

In [59]:
np.linalg.norm(embeddings_array), np.linalg.norm(embeddings_array_doc)

(1.0, 0.9999999999999999)

In [64]:
print("The cosine similarity between the vector for the query and the vector for the document is:\n", embeddings_array.dot(embeddings_array_doc))

The cosine similarity between the vector for the query and the vector for the document is:
 0.9008528895674548


## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

```python
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]
```

Compute the embeddings for the text field, and compute the 
cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

- 0
- 1
- 2
- 3
- 4

Hint: if you put all the embeddings of the text field in one matrix `V` (a single 2-dimensional numpy array), then
computing the cosine becomes a matrix multiplication:

```python
V.dot(q)
```

If this hint is rather confusing you than helping, feel free
to ignore it.

In [65]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [73]:
# Hint: if you put all the embeddings of the text field in one matrix `V` (a single 2-dimensional numpy array), then
# computing the cosine becomes a matrix multiplication:

embeddings_vector = np.array([list(model.embed(doc['text']))[0] for doc in documents])

In [74]:
embeddings_vector

array([[-0.0249525 , -0.0396454 , -0.00437672, ...,  0.06779866,
         0.0377203 , -0.00470724],
       [-0.0594709 , -0.08523984,  0.01292091, ...,  0.09599709,
         0.05420843, -0.00029461],
       [-0.06846454, -0.04079098,  0.04998121, ...,  0.06578071,
         0.02872077, -0.01115215],
       [-0.04640506, -0.02555229,  0.02241224, ...,  0.03410312,
         0.06813592, -0.0064033 ],
       [-0.05394912, -0.04693814,  0.00794725, ...,  0.05402997,
         0.03033385, -0.01254518]])

In [75]:
similarities = embeddings_vector.dot(embeddings_array)

In [76]:
similarities

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

In [77]:
best_idx = np.argmax(similarities)
print("Document index with highest similarity:", best_idx)

Document index with highest similarity: 1


## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of
`question` and `text`:

```python
full_text = doc['question'] + ' ' + doc['text']
``` 

Embed this field and compute the cosine between it and the
query vector. What's the highest scoring document?

- 0
- 1
- 2
- 3
- 4

Is it different from Q3? If yes, why?

In [78]:
embeddings_vector_q4 = np.array([list(model.embed(doc['question']+doc['text']))[0] for doc in documents])

In [80]:
similarities_q4 = embeddings_vector_q4.dot(embeddings_array)
similarities_q4

array([0.85145432, 0.84365942, 0.8408287 , 0.7755158 , 0.80860078])

In [81]:
best_idx_q4 = np.argmax(similarities_q4)
print("Document index with highest similarity:", best_idx_q4)

Document index with highest similarity: 0


## Q5. Selecting the embedding model

Now let's select a smaller embedding model.
What's the smallest dimensionality for models in fastembed?

- 128
- 256
- 384
- 512

One of these models is `BAAI/bge-small-en`. Let's use it.

In [87]:
import pandas as pd
supported_models = (
    pd.DataFrame(TextEmbedding.list_supported_models())
    .sort_values("dim")
    .drop(columns=["sources", "model_file", "additional_files"])
    .reset_index(drop=True)
)
supported_models

,model,description,license,size_in_GB,dim,tasks
0,sentence-transformers/all-MiniLM-L6-v2,"Text embeddings, Unimodal (text), English, 256...",apache-2.0,0.090,384,{}
1,snowflake/snowflake-arctic-embed-xs,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.090,384,{}
2,BAAI/bge-small-en-v1.5,"Text embeddings, Unimodal (text), English, 512...",mit,0.067,384,{}
3,snowflake/snowflake-arctic-embed-s,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.130,384,{}
4,sentence-transformers/paraphrase-multilingual-...,"Text embeddings, Unimodal (text), Multilingual...",apache-2.0,0.220,384,{}
5,BAAI/bge-small-en,"Text embeddings, Unimodal (text), English, 512...",mit,0.130,384,{}
6,BAAI/bge-small-zh-v1.5,"Text embeddings, Unimodal (text), Chinese, 512...",mit,0.090,512,{}
7,Qdrant/clip-ViT-B-32-text,"Text embeddings, Multimodal (text&image), Engl...",mit,0.250,512,{}
8,jinaai/jina-embeddings-v2-small-en,"Text embeddings, Unimodal (text), English, 819...",apache-2.0,0.120,512,{}
9,nomic-ai/nomic-embed-text-v1,"Text embeddings, Multimodal (text, image), Eng...",apache-2.0,0.520,768,{}


In [95]:
# Get the model with the smallest dimensionality
smallest_model = supported_models[supported_models["dim"] == supported_models["dim"].min()].sort_values('size_in_GB')
smallest_model

,model,description,license,size_in_GB,dim,tasks
2,BAAI/bge-small-en-v1.5,"Text embeddings, Unimodal (text), English, 512...",mit,0.067,384,{}
0,sentence-transformers/all-MiniLM-L6-v2,"Text embeddings, Unimodal (text), English, 256...",apache-2.0,0.090,384,{}
1,snowflake/snowflake-arctic-embed-xs,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.090,384,{}
3,snowflake/snowflake-arctic-embed-s,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.130,384,{}
5,BAAI/bge-small-en,"Text embeddings, Unimodal (text), English, 512...",mit,0.130,384,{}
4,sentence-transformers/paraphrase-multilingual-...,"Text embeddings, Unimodal (text), Multilingual...",apache-2.0,0.220,384,{}


## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

```python
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results?
(The score for the first returned record):

- 0.97
- 0.87
- 0.77
- 0.67

In [92]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [93]:
from qdrant_client import QdrantClient, models

In [94]:
client = QdrantClient("http://localhost:6333")

In [96]:
model_handle = 'BAAI/bge-small-en'
EMBEDDING_DIMENSIONALITY = 384
# Define the collection name
collection_name = "zoomcamp-rag-homework"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY, 
        distance=models.Distance.COSINE 
    )
)

points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['question'] + doc['text'], model=model_handle),
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1

In [97]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [99]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [100]:
query = 'I just discovered the course. Can I join now?'

In [110]:
results = search(query)
results

QueryResponse(points=[ScoredPoint(id=449, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [111]:
print("The highest score in the results is: {}".format(results.points[0].score))

The highest score in the results is: 0.8703172
